# 用Agent API 构建

In [ ]:
import requests

# create_tool_calling_agent函数和AgentExecutor类，是本节智能体构建内容的关键。
from langchain.agents import create_tool_calling_agent, tool, AgentExecutor

from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

@tool
def get_weather(loc):
    """
        查询即时天气函数
        :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
        :return：心知天气 API查询即时天气的结果，具体URL请求地址为："https://api.seniverse.com/v3/weather/now.json"
        返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    url = "https://api.seniverse.com/v3/weather/now.json"
    params = {
        "key": "你注册的心知天气api",
        "location": loc,
        "language": "zh-Hans",
        "unit": "c",
    }
    response = requests.get(url, params=params)
    temperature = response.json()
    return temperature['results'][0]['now']

构建提示词模板组件和大模型组件（注意提示词模板中的agent提示符），为保证大模型具备更强的Agent能力

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是天气助手，请根据用户的问题，给出相应的天气信息"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),  # 这部分agnet提示符不需要人工输入，同时也是写死的不可以修改
    ]
)

# 使用 DeepSeek 模型
model = init_chat_model(
    model='deepseek-chat', # deepseek-chat表示调用DeepSeek-v3模型
    model_provider='deepseek',# 模型提供商写deepseek
    api_key="你注册的deepseek api key",
)

In [ ]:
#定义工具
tools = [get_weather]

# 项目中不再使用model.bind()绑定模型组件和工具，
# 而是使用create_tool_calling_agent快速构建代理，代码如下:
agent = create_tool_calling_agent(model, tools, prompt)

In [ ]:
# 使用AgentExecutor构建Agent并快速运行代理
# 也可以使用之前的 chain 构建
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # Verbose表示是否要打印执行细节
response=agent_executor.invoke({"input":"请问今天北京天气怎么样？"})

print(response)

## 并联调用
如果用户同时询问北京和杭州的天气，LangChain Agent会并联查询两个地方的天气

In [ ]:
response=agent_executor.invoke({"input":"请问今天北京和杭州的天气怎么样，哪个城市更热？？"})
print(response)

## 串联调用 + 多 tools

In [ ]:
@tool
def write_file(content):
    """
    将指定内容写入本地文件。
    :param content: 必要参数，字符串类型，用于表示需要写入文档的具体内容。
    :return：是否成功写入
    """
    with open('res.txt', 'w', encoding='utf-8') as f:
        f.write(content)
    return "已成功写入本地文件。"

# 定义多个工具
tools = [get_weather, write_file]

# 修改prompt，使模型知道它已经具备了写入能力
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是天气助手，请根据用户的问题，给出相应的天气信息,并具备将结果写入文件的能力"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),  # 这部分agnet提示符不需要人工输入，同时也是写死的不可以修改
    ]
)

agent = create_tool_calling_agent(model, tools, prompt)

# 使用AgentExecutor运行当前Agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # Verbose表示是否要打印细节信息


response=agent_executor.invoke({"input":"查一下北京和杭州现在的温度，并将结果写入本地的文件中。"})
print(response)

LangChain内置搜索工具 Tavily，每月有免费额度

pip install langchain-tavily

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
# 导入 tavily 搜索工具
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)

tools = [search]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一名助人为乐的助手，并且可以调用工具进行网络搜索，获取实时信息。"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
        # placeholder 只是一个占位符
        # Agent 会用agent_scratchpad储存中间步骤(intermediate steps)相关资讯，
        # 例如Agent 此次的任务已经执行过哪些步骤以及该步骤的输出等等，
        # 这样Agent 才会知道哪些步骤已经做过，我们不需要对它做任何事情。
        # 例如还可以用来存储对话结果：("placeholder", "{chat_history}"),
        # 可以使用多个 placeholder

    ]
)
# 初始化模型
# 使用 DeepSeek 模型
model = init_chat_model(
    model='deepseek-chat', # deepseek-chat表示调用DeepSeek-v3模型
    model_provider='deepseek',# 模型提供商写deepseek
    api_key="你注册的api key",
)
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "请问苹果2025WWDC发布会召开的时间是？"})